In [ ]:
# Install necessary packages
!pip install tensorflow==2.7.0  # Ensure to use compatible version
!pip install tf_slim
!pip install pycocotools

In [ ]:
# Clone the TensorFlow Models repository
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=
cp object_detection/packages/tf2/setup.py
python -m pip install

In [ ]:
# importing libraries
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.utils import config_util

In [ ]:
# Path to your pipeline config file
pipeline_config_path = 'path' # adding file path

# Path to your checkpoint directory
model_dir = 'path' # adding file path

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=True)

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=True)

In [ ]:
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

In [ ]:
# Function to preprocess an image
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (640, 640))  # Resize to match model's expected sizing
    image = tf.expand_dims(image, axis=0)
    return image

In [ ]:
# Function to detect license plates in an image
def detect_license_plate(image_path):
    image_np = preprocess_image(image_path)
    input_tensor = tf.convert_to_tensor(image_np)
    detections = detection_model(input_tensor)
    detection_boxes = detections['detection_boxes'][0].numpy()
    detection_scores = detections['detection_scores'][0].numpy()
    detection_classes = detections['detection_classes'][0].numpy().astype(int)

    # Filter detections to only include license plates
    license_plate_boxes = detection_boxes[detection_classes == 1]  # Assuming class 1 is license plate

    return license_plate_boxes

In [ ]:
# Example usage:
image_path = 'path' # adding file path
license_plate_boxes = detect_license_plate(image_path)
print("Detected license plate bounding boxes:", license_plate_boxes)

In [ ]:
# Install necessary packages
!pip install torch torchvision matplotlib opencv-python

import torch
import torchvision.transforms as transforms
import torchvision.models as models
import cv2

# Define a simple CNN model for character recognition
class CharacterRecognizerCNN(torch.nn.Module):
    def __init__(self):
        super(CharacterRecognizerCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = torch.nn.Linear(64 * 8 * 8, 128)
        self.fc2 = torch.nn.Linear(128, 36)  # Assuming 36 classes for alphanumeric characters

    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 64 * 8 * 8)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load pretrained model weights
model = CharacterRecognizerCNN()
model.load_state_dict(torch.load('path/to/your/model.pth'))
model.eval()

# Function to preprocess license plate image
def preprocess_license_plate(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 64))  # Resize to match model's expected input size
    image = transforms.functional.to_tensor(image)
    image = torch.unsqueeze(image, 0)  # Add batch dimension
    return image

# Function to recognize characters on a license plate
def recognize_characters(image_path):
    image = preprocess_license_plate(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
        predicted_char = chr(predicted.item() + 65)  # Convert numeric class to character (assuming class 0-25 are A-Z)
    return predicted_char

# Example usage:
license_plate_image_path = 'path/to/your/license_plate_image.jpg'
predicted_character = recognize_characters(license_plate_image_path)
print("Predicted character on license plate:", predicted_character)